# Assignment 6

### Part A

In [18]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [19]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [20]:
def exp_return(data, P):
    exp_returns = (data['Price in 1 Year']/P) - 1
    result = sum(exp_returns * data['Probability'])
    return result
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [21]:
def volatility(data, P):
    exp_returns = (data['Price in 1 Year'] /P) - 1
    expected_return = exp_return(data, P)
    variances = (exp_returns - expected_return) ** 2
    expected_variance = sum(variances * data['Probability'])
    result = np.sqrt(expected_variance)
    return result

    # Write your code here
    # volatility(data,40) should output 0.21301408404140795

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [22]:
def interval(data, P):
    std = volatility(data, P)/(3** (0.5))
    expected_return = exp_return(data, P)
    lower_bound =  expected_return - (std*1.96)
    upper_bound =  expected_return + (std*1.96)

    return [lower_bound, upper_bound]

    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [23]:
def exp_return_capm(beta, market_return, rf):
    result = beta * (market_return - rf) + rf
    return result

    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [24]:
def clean_data(path):
    df = pd.read_csv(path)
    
    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    
    df.dropna(subset=['RET'], inplace=True)
    
    checker = df['PERMNO'].value_counts()
    
    valid_stat = checker[checker >= 500].index

    df = df[df['PERMNO'].isin(valid_stat)]

    return df[['PERMNO', 'date', 'CUSIP', 'RET']]

    # This should yield data with 403316 rows × 4 columns


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [25]:
def avg_return(data):

    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    
    result = data.groupby('CUSIP', as_index=False)['RET'].mean()
    
    result.rename(columns={'RET': 'Average Return'}, inplace=True)
    
    return result

    # Write your code here
    # This should yield data with 679 rows × 2 columns

In [26]:
def security_volatilities(data):
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    
    result = data.groupby('CUSIP', as_index=False)['RET'].std()
    
    result.rename(columns={'RET': 'Volatility'}, inplace=True)
    
    return result
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [27]:
def correlation(ret_data, vol_data):

    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    
    correlation_matrix = merged_data[['Average Return', 'Volatility']].corr()
    
    result = correlation_matrix.loc['Average Return', 'Volatility']
    
    return result
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614